Imports

In [1372]:
from random import shuffle
from queue import PriorityQueue
import numpy as np
from typing import Protocol, Dict, List, Iterator, Tuple, TypeVar, Optional
from __future__ import annotations
import heapq

The graph interface gets the neighbors for each graph location.

In [1373]:
T = TypeVar('T')
Location = TypeVar('Location')
class Graph(Protocol):
    def neighbors(self, id: Location) -> List[Location]: pass

Setting up a priority queue system for what nodes the algorithm should visit.

In [1374]:
class PriorityQueue:
    def __init__(self):
        self.elements: List[Tuple[float, T]] = []
    
    def empty(self) -> bool:
        return not self.elements
    
    def put(self, item: T, priority: float):
        heapq.heappush(self.elements, (priority, item))
    
    def get(self) -> T:
        return heapq.heappop(self.elements)[1]

Creating up a grid calculating neighbors in the neighbors function.

In [1375]:
GridLocation = Tuple[int, int]
class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.walls: List[GridLocation] = []
    
    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def passable(self, id: GridLocation) -> bool:
        return id not in self.walls
    
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        results = filter(self.passable, results)
        return results

Interface for a weighted graph.

In [1376]:
class WeightedGraph(Graph):
    def cost(self, from_id: Location, to_id: Location) -> float: pass

Implementation of the interface with a grid. Weighted graphs adds a cost moving along an edge.

In [1377]:
class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int):
        super().__init__(width, height)
        self.weights: Dict[GridLocation, float] = {}
    
    def cost(self, from_node: GridLocation, to_node: GridLocation) -> float:
        return self.weights.get(to_node, 1)

Adds a heuristic. This gives us the sum of the absolute values the estimated distance to the goal and the goals distance to the start point. 

In [1378]:
def heuristic(a: GridLocation, b: GridLocation) -> float:
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

Draws the tiles and has a couple of different style choices.

In [1379]:
def draw_tile(graph, id, style):
    r = " . "
    if 'number' in style and id in style['number']: r = " %-2d" % style['number'][id]
    if 'point_to' in style and style['point_to'].get(id, None) is not None:
        (x1, y1) = id
        (x2, y2) = style['point_to'][id]
        if x2 == x1 + 1: r = " > "
        if x2 == x1 - 1: r = " < "
        if y2 == y1 + 1: r = " v "
        if y2 == y1 - 1: r = " ^ "
    if 'path' in style and id in style['path']:   r =" @ "
    if 'start' in style and id == style['start']: r = " S "
    if 'goal' in style and id == style['goal']:   r = " G "
    if id in graph.walls: r = "###"
    return r

Draws the grids itself.

In [1380]:
def draw_grid(graph, **style):
    print("___" * graph.width)
    for y in range(graph.height):
        for x in range(graph.width):
            print("%s" % draw_tile(graph, (x, y), style), end="")
        print()
    print("~~~" * graph.width)

Example diagram created to show the basic idea.

In [1381]:
diagram_example = GridWithWeights(10, 10)
diagram_example.walls = [(1, 7), (1, 8), (2, 7), (2, 8), (3, 7), (3, 8)]
diagram_example.weights = {loc: 5 for loc in [(3, 4), (3, 5), (4, 1), (4, 2),
                                       (4, 3), (4, 4), (4, 5), (4, 6),
                                       (4, 7), (4, 8), (5, 1), (5, 2),
                                       (5, 3), (5, 4), (5, 5), (5, 6),
                                       (5, 7), (5, 8), (6, 2), (6, 3),
                                       (6, 4), (6, 5), (6, 6), (6, 7),
                                       (7, 3), (7, 4), (7, 5)]}

Shows the solution.

In [1382]:
def reconstruct_path(came_from: Dict[Location, Location],
                     start: Location, goal: Location) -> List[Location]:
    current: Location = goal
    path: List[Location] = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

The a* algorithm itself.

In [1383]:
def a_star_search(graph: WeightedGraph, start: Location, goal: Location):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: Dict[Location, Optional[Location]] = {}
    cost_so_far: Dict[Location, float] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: Location = frontier.get()
        
        if current == goal:
            break
        
        for next in graph.neighbors(current):
            new_cost = cost_so_far[current] + graph.cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

Example where you can see how the algorithm works when it is weighted. It will prefer the lowest cost.

In [1384]:
start, goal = (1, 4), (8, 3)
came_from, cost_so_far = a_star_search(diagram_example, start, goal)
draw_grid(diagram_example, start=start, goal=goal)
print()
draw_grid(diagram_example, point_to=came_from, start=start, goal=goal)
print()
draw_grid(diagram_example, number=cost_so_far, start=start, goal=goal)
print()
draw_grid(diagram_example, path=reconstruct_path(came_from, start=start, goal=goal))

______________________________
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  G  . 
 .  S  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

______________________________
 v  v  v  v  <  <  <  <  <  < 
 v  v  v  v  <  ^  ^  <  <  < 
 v  v  v  v  <  <  ^  ^  <  < 
 >  v  <  <  <  <  .  ^  G  . 
 >  S  <  <  <  .  .  .  .  . 
 ^  ^  ^  <  <  .  .  .  .  . 
 ^  ^  ^  <  <  .  .  .  .  . 
 ^ ######### .  .  .  .  .  . 
 . ######### .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  . 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

______________________________
 5  4  5  6  7  8  9  10 11 12
 4  3  4  5  10 13 10 11 12 13
 3  2  3  4  9  14 15 12 13 14
 2  1  2  3  8  13 .  17 G  . 
 1  S  1  6  11 .  .  .  .  . 
 2  1  2  7  12 .  .  .  .  . 
 3  2  3  4  9  .  .  .  .  . 
 4 ###

Creating up a maze from the code we got on learn.

In [1385]:
def make_maze(w = 50, h = 25):
    vis = [[0] * w + [1] for _ in range(h)] + [[1] * (w + 1)]
    ver = [["10"] * w + ['1'] for _ in range(h)] + [[]]
    hor = [["11"] * w + ['1'] for _ in range(h + 1)]
 
    def walk(x, y):
        vis[y][x] = 1
 
        d = [(x - 1, y), (x, y + 1), (x + 1, y), (x, y - 1)]
        shuffle(d)
        for (xx, yy) in d:
            if vis[yy][xx]: continue
            if xx == x: hor[max(y, yy)][x] = "10"
            if yy == y: ver[y][max(x, xx)] = "00"
            walk(xx, yy)
 
    walk(np.random.randint(w), np.random.randint(h))
 
    s = ""
    for (a, b) in zip(hor, ver):
        s += ''.join(a + ['\n'] + b + ['\n'])
        
    M=[]
    for line in s.split("\n"):
        if line!="":
            R=[]
            for e in line:
                R.append(int(e))
            M.append(R)
    return M
 
maze= np.array(make_maze(5,5))


Creating a function to reformat the maze.

In [1386]:
def format_maze(maze):
    row = -1
    wall = []
    road = []
    for i in maze:
        row=row + 1
        col = 0
        for l in i:
            if l == 1:
                wall.append((row, col))
                col = col+1
            if l == 0:
                road.append((row, col))
                col = col+1
    return wall, road

Reformat the maze.

In [1387]:
wall = []
road = []
wall, road = format_maze(maze)

Create a new diagram based on the maze we created.

In [1388]:
diagram = GridWithWeights(len(maze), len(maze))
diagram.walls = wall
diagram.weights = {loc: 5 for loc in road}

Here we display the maze. As you can see first we display the maze

In [1389]:
start, goal = (1, 1), (9, 9)
came_from, cost_so_far = a_star_search(diagram, start, goal)
print("Actual maze")
draw_grid(diagram, start=start,goal=goal)
print()
print("Maze pointing from goal to start, if you follow the arrows from G you will arrive to the start with the lowest path.")
draw_grid(diagram, point_to=came_from, start=start, goal=goal)
print()
print("Maze using costs.")
draw_grid(diagram, number=cost_so_far, start=start, goal=goal)
print()
print("Shortest path. This is the solution the algorithm has given us.")
draw_grid(diagram, path=reconstruct_path(came_from, start=start, goal=goal))

Actual maze
_________________________________
#################################
### S  .  . ### .  .  .  .  . ###
### . ### . ######### . ### . ###
### . ### .  .  . ### . ### . ###
############### . ### . #########
### .  .  .  .  . ### .  .  . ###
### . ##################### . ###
### .  .  . ### .  .  .  .  . ###
######### . ######### . ### . ###
### .  .  .  .  .  .  . ### G ###
#################################
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Maze pointing from goal to start, if you follow the arrows from G you will arrive to the start with the lowest path.
_________________________________
#################################
### S  <  < ### .  .  .  .  . ###
### ^ ### ^ ######### . ### . ###
### ^ ### ^  <  < ### . ### . ###
############### ^ ### . #########
### >  >  >  >  ^ ### .  .  v ###
### ^ ##################### v ###
### ^  <  < ### >  >  v  <  < ###
######### ^ ######### v ### ^ ###
### >  >  ^  <  <  <  < ### G ###
#################################
~~~~~~~~~~~~~~~~~~~~